In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import os
import random
from datasets import load_dataset
from datetime import datetime
import argparse
import sys
import json
from tabulate import tabulate

from mypkg.pipeline.setup.dataset import (
    load_raw_dataset,
    filter_by_industry,
    filter_by_demographics,
    prepare_dataset_for_model,
)
from mypkg.main_paper_dataset import balanced_downsample

# Set random seed for reproducibility
random_seed = 42
random.seed(random_seed)

# Setup cache directory for output files
cache_dir = os.path.join(os.getcwd(), "cache")
os.makedirs(cache_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")



In [ ]:
# Cell 3: Function to load the datasets
def load_datasets(industry="INFORMATION-TECHNOLOGY", 
                 use_anthropic_dataset=False,
                 dataset_type="implicit",
                 downsample_to=None):
    """
    Load and prepare datasets for analysis
    
    Args:
        industry: Industry to filter for (only for non-Anthropic dataset)
        use_anthropic_dataset: Whether to use the Anthropic dataset
        dataset_type: Type of Anthropic dataset to use
        downsample_to: Number of samples to downsample to (if None, use all)
        
    Returns:
        DataFrame containing the loaded and filtered dataset
    """
    print("Loading dataset...")
    
    if use_anthropic_dataset:
        print(f"Loading Anthropic dataset ({dataset_type})...")
        dataset = load_dataset("Anthropic/discrim-eval", dataset_type)
        df = dataset["train"]
        filtered_df = [item for item in df if item["decision_question_id"] == 16]
        df = pd.DataFrame(filtered_df)
    else:
        print("Loading custom dataset...")
        df = load_raw_dataset()
        
        if industry:
            print(f"Filtering for industry: {industry}")
            df = filter_by_industry(df, industry)
        else:
            print("No industry filter applied.")
    
    # Apply downsampling if requested
    if downsample_to is not None and downsample_to > 0:
        print(f"Downsampling to {downsample_to} samples...")
        df = balanced_downsample(df, downsample_to, random_seed)
    
    print(f"Dataset loaded with {len(df)} entries")
    
    return df

# Cell 4: Example usage
# You can run this to test loading the datasets
if __name__ == "__main__":
    # Example: Load standard dataset for IT industry
    standard_df = load_datasets(industry="INFORMATION-TECHNOLOGY", downsample_to=100)
    
    # Example: Load Anthropic dataset
    # anthropic_df = load_datasets(use_anthropic_dataset=True, dataset_type="explicit")
    anthropic_df = load_datasets(use_anthropic_dataset=True, dataset_type="implicit")
    
    # Display basic info about the datasets
    print("\nStandard Dataset Info:")
    print(f"Shape: {standard_df.shape}")
    print("Columns:", standard_df.columns.tolist())
    
    print("\nAnthropic Dataset Info:")
    print(f"Shape: {anthropic_df.shape}")
    print("Columns:", anthropic_df.columns.tolist())

In [ ]:
def filter_anthropic_df(df: pd.DataFrame) -> pd.DataFrame:
    # Convert to DataFrame if not already
    df = pd.DataFrame(df)
    
    # Filter for decision question ID
    df = df[df["decision_question_id"] == 16]
    
    # Filter for race (white or black only)
    df = df[df["race"].str.lower().isin(["white", "black"])]
    
    return df

def modify_anthropic_filled_templates(df):
    """Change the anthropic templates to match the hiring bias prompts"""
    def modify(text):
        if "accountant" not in text:
            return text
        # Replace role
        text = text.replace("accounting", "")
        text = text.replace("accountant", "IT consultant")
        # Remove the word 'financial'
        text = text.replace("financial ", "")
        # Change job offer question
        text = text.replace(
            "make the applicant a job offer",
            "offer the applicant an interview"
        )
        return text

    df = df.copy()
    df['filled_template'] = df['filled_template'].apply(modify)
    return df

anthropic_df = filter_anthropic_df(anthropic_df)
anthropic_df = modify_anthropic_filled_templates(anthropic_df)
print(anthropic_df.head())
print(len(anthropic_df))

In [ ]:
for i in range(len(anthropic_df)):
    print(anthropic_df.iloc[i]['filled_template'])
    print("\n\n")